In [ ]:
import requests
import os

def get_cid_from_cas(cas_number):
    """Zwraca pierwszy znaleziony CID z PubChem dla danego numeru CAS."""
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{cas_number}/cids/JSON"
    r = requests.get(url)
    if r.status_code == 200 and "IdentifierList" in r.json():
        return r.json()["IdentifierList"]["CID"][0]
    else:
        return None

def get_substance_name(cid):
    """Pobiera preferowaną nazwę substancji z PubChem."""
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/description/JSON"
    r = requests.get(url)
    if r.status_code == 200:
        try:
            return r.json()['InformationList']['Information'][0]['Title'].replace(" ", "_")
        except (KeyError, IndexError):
            return None
    return None

def download_sdf(cid, name, cas_number):
    """
    Pobiera plik SDF w wersji 3D, jeśli jest dostępny; w przeciwnym razie pobiera 2D.
    Nazwa pliku: <Name>_<CAS>_<CID>_<3D/2D>.sdf
    """
    urls = [
        (f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/SDF?record_type=3d", "3D"),
        (f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/SDF?record_type=2d", "2D")
    ]
    for url, suffix in urls:
        response = requests.get(url)
        if response.status_code == 200 and len(response.content) > 100:
            filename = f"{name}_{cas_number}_{cid}_{suffix}.sdf"
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"Pobrano plik: {filename}")
            return filename
    print("Brak pliku SDF dla CID:", cid)
    return None

def cas_to_pubchem_sdf(cas_list):
    """
    Dla listy numerów CAS znajduje nazwy substancji oraz pobiera pliki SDF.
    """
    results = []
    for cas in cas_list:
        print(f"Przetwarzam CAS: {cas}")
        cid = get_cid_from_cas(cas)
        if cid is None:
            print(f"Nie znaleziono CID dla CAS: {cas}")
            continue
        name = get_substance_name(cid)
        if name is None:
            name = "Unknown"
        sdf_file = download_sdf(cid, name, cas)
        results.append({"CAS": cas, "CID": cid, "Name": name, "SDF": sdf_file})
    return results

# Przykładowe użycie:
cas_numbers = ["50-00-0", "64-17-5", "67-56-1"]  # formaldehyd, etanol, metanol
results = cas_to_pubchem_sdf(cas_numbers)
print("Podsumowanie wyników:", results)